# Mercari Price Prediction ML

In [1]:
import scipy as sp
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
import re
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_log_error
from sklearn.linear_model import RidgeCV, Ridge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import scipy

#### Pulling in premade Kaggle competition data.

In [2]:
train = pd.read_csv("train.tsv", sep='\t', header=0)
train.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


In [3]:
test = pd.read_csv("test.tsv", sep='\t', header=0)
test.head()

,test_id,name,item_condition_id,category_name,brand_name,shipping,item_description
0,0,"Breast cancer ""I fight like a girl"" ring",1,Women/Jewelry/Rings,NaN,1,Size 7
1,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers",1,Other/Office supplies/Shipping Supplies,NaN,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers Lined..."
2,2,Coach bag,1,Vintage & Collectibles/Bags and Purses/Handbag,Coach,1,Brand new coach bag. Bought for [rm] at a Coac...
3,3,Floral Kimono,2,Women/Sweaters/Cardigan,NaN,0,-floral kimono -never worn -lightweight and pe...
4,4,Life after Death,3,Other/Books/Religion & Spirituality,NaN,1,Rediscovering life after the loss of a loved o...


#### Need to remove null values from item_description, as they would interfere with the algorithm.

In [4]:
train = train[~train['item_description'].isnull()]
test = test[~test['item_description'].isnull()]

#### Combining datasets for train and test together so that all function performed in preparing the data will not skew the size of the matrix

In [5]:
data = pd.concat([train, test], 0)
train_rows = train.shape[0]
print(train_rows)

1482531


Replacing 'No description yet' to no_desc for tfidf vectorizer to count these as stop words. Also transforming category_name into two distinct categories to introduce two features for the matrix

In [6]:
data.item_description = data.item_description.str.replace('No description yet', 'no_desc')
data['primary_cat'] = data.category_name.str.extract('([^/]+)/[^/]+/[^/]+')
data['secondary_cat'] = data.category_name.str.extract('[^/]+/([^/]+/[^/]+)')
data = data.apply(lambda x: x.astype(str).str.lower())

c:\users\kj.park\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  
c:\users\kj.park\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
data.head()

,brand_name,category_name,item_condition_id,item_description,name,price,shipping,test_id,train_id,primary_cat,secondary_cat
0,nan,men/tops/t-shirts,3,no_desc,mlb cincinnati reds t shirt size xl,10.0,1,nan,0.0,men,tops/t-shirts
1,razer,electronics/computers & tablets/components & p...,3,this keyboard is in great condition and works ...,razer blackwidow chroma keyboard,52.0,0,nan,1.0,electronics,computers & tablets/components & parts
2,target,women/tops & blouses/blouse,1,adorable top with a hint of lace and a key hol...,ava-viv blouse,10.0,1,nan,2.0,women,tops & blouses/blouse
3,nan,home/home décor/home décor accents,1,new with tags. leather horses. retail for [rm]...,leather horse statues,35.0,1,nan,3.0,home,home décor/home décor accents
4,nan,women/jewelry/necklaces,1,complete with certificate of authenticity,24k gold plated rose,44.0,0,nan,4.0,women,jewelry/necklaces


In [8]:
text_features = ['name', 'brand_name', 'category_name', 'primary_cat', 'secondary_cat', 'item_description']
for t in text_features:
    data[t].replace(regex=True,inplace=True,to_replace=r'\W',value=r' ')
    
data.head()

,brand_name,category_name,item_condition_id,item_description,name,price,shipping,test_id,train_id,primary_cat,secondary_cat
0,nan,men tops t shirts,3,no_desc,mlb cincinnati reds t shirt size xl,10.0,1,nan,0.0,men,tops t shirts
1,razer,electronics computers tablets components p...,3,this keyboard is in great condition and works ...,razer blackwidow chroma keyboard,52.0,0,nan,1.0,electronics,computers tablets components parts
2,target,women tops blouses blouse,1,adorable top with a hint of lace and a key hol...,ava viv blouse,10.0,1,nan,2.0,women,tops blouses blouse
3,nan,home home décor home décor accents,1,new with tags leather horses retail for rm ...,leather horse statues,35.0,1,nan,3.0,home,home décor home décor accents
4,nan,women jewelry necklaces,1,complete with certificate of authenticity,24k gold plated rose,44.0,0,nan,4.0,women,jewelry necklaces


In [9]:
data['price'] = pd.to_numeric(data.price, errors = 'ignore')
data['item_condition_id'] = pd.to_numeric(data.item_condition_id, errors = 'ignore')
data['shipping'] = pd.to_numeric(data.shipping, errors = 'ignore')
data['item_description'] = data['item_description'].fillna('')

In [10]:
data['brand_name'] = data['brand_name'].replace([np.nan,'nan'], 'negative', regex=True)
data['brand_name'] = data['brand_name'].str.replace('\s+', '')  # in case there are multiple white spaces
data['brand_name'] = 'brand_' + data['brand_name'].astype(str)


In [11]:
data.head()

,brand_name,category_name,item_condition_id,item_description,name,price,shipping,test_id,train_id,primary_cat,secondary_cat
0,brand_negative,men tops t shirts,3,no_desc,mlb cincinnati reds t shirt size xl,10.0,1,nan,0.0,men,tops t shirts
1,brand_razer,electronics computers tablets components p...,3,this keyboard is in great condition and works ...,razer blackwidow chroma keyboard,52.0,0,nan,1.0,electronics,computers tablets components parts
2,brand_target,women tops blouses blouse,1,adorable top with a hint of lace and a key hol...,ava viv blouse,10.0,1,nan,2.0,women,tops blouses blouse
3,brand_negative,home home décor home décor accents,1,new with tags leather horses retail for rm ...,leather horse statues,35.0,1,nan,3.0,home,home décor home décor accents
4,brand_negative,women jewelry necklaces,1,complete with certificate of authenticity,24k gold plated rose,44.0,0,nan,4.0,women,jewelry necklaces


data.shape

In [12]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
import nltk
nltk.download('wordnet')
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

tfidf = TfidfVectorizer(tokenizer=LemmaTokenizer(),
                                strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                max_df = .9,
                                min_df = .005,
                                max_features=50000,
                                ngram_range=(1, 2))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kj.park\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [13]:
text = (data['brand_name'] + ' ' +
          data['item_description'] + ' ' + data['primary_cat'] + ' ' + 
          data['secondary_cat']).values

In [14]:
text

array(['brand_negative no_desc men tops t shirts',
       'brand_razer this keyboard is in great condition and works like it came out of the box  all of the ports are tested and work perfectly  the lights are customizable via the razer synapse app on your pc  electronics computers   tablets components   parts',
       'brand_target adorable top with a hint of lace and a key hole in the back  the pale pink is a 1x  and i also have a 3x available in white  women tops   blouses blouse',
       ...,
       'brand_negative new  free shipping basstop case electronics cell phones   accessories cases  covers   skins',
       'brand_negative floral kimono  tropical print  open front  hi low design  great for layering or as a cover up  women swimwear cover ups',
       'brand_negative 2 floral scrub tops  worn less than 5 times each  brown has a belt tie  women tops   blouses t shirts'], dtype=object)

In [15]:
text = tfidf.fit_transform(text)

In [16]:
text

<2175890x881 sparse matrix of type '<class 'numpy.float64'>'
	with 38554085 stored elements in Compressed Sparse Row format>

#### Separating the training and test sets again after massaging the data for the distinct train and test sets with the train_rows mask. Also creating the y value with train['price'] as log as we had found during the exploratory data analysis portion of the project

In [17]:
dummies = scipy.sparse.csr_matrix(pd.get_dummies(data[['item_condition_id', 'shipping']], sparse = True).values)

In [18]:
X = scipy.sparse.hstack((text, dummies)).tocsr()

In [19]:
train = X[:train_rows]
test = X[train_rows:]

In [20]:
train_price = data['price'][:train_rows]
train_price = train_price.astype(float)

In [21]:
y= np.log1p(train_price)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.4, random_state=42)

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

for Model in [Ridge, RandomForestRegressor]:
    model = Model()
    print('%s: %s' % (Model.__name__,
                      cross_val_score(model, X_train, y_train).mean()))

In [23]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor as lgb
param_grid = {'learning_rate': [0.01, 0.1, 0.05, .5, 1], 'n_estimators': [20, 40, 60], 'num_leaves': [30, 35, 40]}
print("starting grid search ......")
optimized_GBM = GridSearchCV(lgb(objective='regression', verbose=200),
                             param_grid = param_grid,
                             cv=5,
                             n_jobs=-1
                             )

optimized_GBM.fit(X_train, y_train)

starting grid search ......


GridSearchCV(cv=5, error_score='raise',
       estimator=LGBMRegressor(boosting_type='gbdt', colsample_bytree=1.0, learning_rate=0.1,
       max_bin=255, max_depth=-1, min_child_samples=10, min_child_weight=5,
       min_split_gain=0.0, n_estimators=10, n_jobs=-1, num_leaves=31,
       objective='regression', random_state=0, reg_alpha=0.0,
       reg_lambda=0.0, silent=True, subsample=1.0, subsample_for_bin=50000,
       subsample_freq=1, verbose=200),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'learning_rate': [0.01, 0.1, 0.05, 0.5, 1], 'n_estimators': [20, 40, 60], 'num_leaves': [30, 35, 40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [28]:

optimized_GBM.score(X_test, y_test)

0.43510985446923478

In [36]:
from sklearn.linear_model import RidgeCV

model = Ridge()
params={'alpha': [.0001,.00001]}
gscv = GridSearchCV(estimator=model,
                  param_grid=params,
                  verbose=200)

In [43]:
from sklearn.metrics import mean_squared_error

ridge = RidgeCV(alphas=(0.1, 1.0, 10.0), fit_intercept=True, 
        normalize=False, scoring='mean_squared_error', cv=5, gcv_mode='auto',
        store_cv_values=False)


In [ ]:
ridge.fit(X_train, y_train)

c:\users\kj.park\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
c:\users\kj.park\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
params={'n_estimators': [5,10,15,20,25]}
gscv = GridSearchCV(estimator=model,
                  param_grid=params,
                  scoring='mean_squared_error',
                  n_jobs=-1,
                  cv=5,
                  verbose=200)

gscv.fit(X_train, y_train)
#best estimators
print("Best Estimator: ", gscv.best_estimator_)
#printing best scores
print("Best Score: ", gscv.best_score_)
#printing best parameters for optimal parameter tuning
print("Best Parameters: ", gscv.best_params_)


In [ ]:
predictions = gscv.predict(X_test)
#Would need to do a root mean squared error on the predictions vs real y_test.
from sklearn.metrics import mean_squared_error
from math import sqrt

rmsle = sqrt(mean_squared_error(y_test, predictions))
